In [51]:
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict

In [15]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file

In [16]:
data_only_drive = '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2'  # external date only drive
data_dir = os.getenv('FINANCE_DATA') #internal folder with finance data 
symbols = [s for s in os.listdir(data_dir) if s.endswith('.L')]

In [91]:
labels = os.path.join(data_dir,'Labels')

idxSymbol = 1 #0,1,2, in our case - so you pick a symbol here
print("you picked symbol: ",symbols[idxSymbol])  

#based on the symbol select the features path
symbolFeaturesPath = os.path.join(data_dir,symbols[idxSymbol],'MODEL_BASED') 
# path all the list of dates we have features for
#select the Labels path as well
symbolLabelsPath = os.path.join(labels, symbols[idxSymbol],'NON_DIRECTIONAL')

symbolLabelsDatesList = [fileDate.split(".")[0] for fileDate in sorted(os.listdir(symbolLabelsPath))]
#strip out all the features dates, for the particular symbol. 
#so far we have selcected a symbol, picked the directory of features and labels for that symbol (features being the HMM files),
# and split the labels files by date

symbolFeaturesListDates = os.listdir(symbolFeaturesPath) 

you picked symbol:  PRU.L


In [115]:
for idx, featuresDate in enumerate(symbolFeaturesListDates): #got through all the features dates
    #create a features path
    featuresDatePath = (os.path.join(symbolFeaturesPath, featuresDate))
    labelsFeaturesDatePaths = [(os.listdir(featuresDatePath)[date].split("_")[5]) for date in os.listdir(featuresDatePath)]

TypeError: list indices must be integers or slices, not str

In [120]:
[os.listdir(featuresDatePath)[x].split("_")[5] for x in os.listdir(featuresDatePath)]


TypeError: list indices must be integers or slices, not str

In [85]:


#list of all the directories each containing the forward features for one HMM day

# to pick a path for features, select a directory from above, within the FeaturesPath, and then look inside.
# so what you have done is pick an HMM model date and then the features inside it.
os.listdir(os.path.join(symbolFeaturesPath, symbolFeaturesListDates[0]))


['PRU.L_3_states_features_date:_20180418_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180420_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180419_now:_20181229_.pickle']

In [87]:
os.path.join(symbolLabelsPath, )


['20170116',
 '20170117',
 '20170118',
 '20170119',
 '20170120',
 '20170123',
 '20170124',
 '20170125',
 '20170126',
 '20170127',
 '20170130',
 '20170131',
 '20170301',
 '20170703',
 '20170704',
 '20170705',
 '20170706',
 '20170707',
 '20170710',
 '20170711',
 '20170712',
 '20170713',
 '20170714',
 '20170717',
 '20170718',
 '20170719',
 '20170720',
 '20170721',
 '20170724',
 '20170725',
 '20170726',
 '20170727',
 '20170728',
 '20170731',
 '20170801',
 '20170802',
 '20170803',
 '20170804',
 '20170807',
 '20170808',
 '20170809',
 '20170810',
 '20170811',
 '20170814',
 '20170815',
 '20170816',
 '20170817',
 '20170818',
 '20170821',
 '20170822',
 '20170823',
 '20170824',
 '20170825',
 '20170829',
 '20170830',
 '20170831',
 '20180201',
 '20180202',
 '20180205',
 '20180206',
 '20180207',
 '20180208',
 '20180209',
 '20180212',
 '20180213',
 '20180214',
 '20180215',
 '20180216',
 '20180219',
 '20180220',
 '20180221',
 '20180222',
 '20180223',
 '20180226',
 '20180227',
 '20180228',
 '20180403',

In [39]:
print('-------------')



symbolfeaturesPath = os.path.join(os.path.join(data_dir,symbols[idxSymbol],'MODEL_BASED'), featuresDateList[idxDate])

#for each of the featuresDateItems, I have a set of features that needs to get matched with a set of labels and used to train an MKLSVM
print(featuresDateItemsPath)
print('---------------')
listFeaturesPickle = os.listdir(featuresDateItemsPath) #all the features!


-------------
/home/ak/FinData/PRU.L/MODEL_BASED/20170713
---------------


In [38]:
idxDate= 3 #as an example, enumerate the above list when parsing thru
print(featuresDateList[idxDate])
os.listdir(featuresDateItems)

20170713


['PRU.L_3_states_features_date:_20180403_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180404_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180208_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180412_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180215_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180226_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180227_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180213_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180406_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180409_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180416_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180228_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180411_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180417_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180212_now:_20181229_.pickle',
 'PRU.L_3_states_features

In [6]:
idxFeatures = 0 #placeholder for enumerator
featuresTuple = pickle.load(open(os.path.join(featuresDateItems,listFeaturesPickle[idxFeatures]), "rb"), encoding='latin1')

In [26]:
featuresDateItems

'/home/ak/FinData/PRU.L/MODEL_BASED/20180202'

In [7]:
featuresArray = np.asarray(pd.concat([featuresTuple[0], featuresTuple[1],\
                                                 featuresTuple[2], featuresTuple[3]], axis=1, sort=False).fillna(0))

In [8]:
labelsDate = os.path.splitext(listFeaturesPickle[idxDate])[0].split("_")[5]
#take the featuresTuple file, pick the first element and split it using "_", then pick the data the features were created

symbolLabelDir = os.path.join(labels,symbols[idxSymbol],'NON_DIRECTIONAL') #path that has all the labels for the particular symbol, i.e all the dates
#each day is a set of labels

In [9]:
labelsFile = os.path.join(symbolLabelDir,labelsDate+".csv") # formulate the path for (as an example) the specific date we dealt with

In [10]:
labelsDf= pd.read_csv(labelsFile).fillna(0) #load the dataframe and replace the NAs with zero (this is small cheat)
labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] #just retain the column of labels please

In [11]:
#formulate the problem
Y_all = labels
X_all = featuresArray

In [12]:
X_all.shape[0] == Y_all.shape[0] #check for dimensions, if False, there is a problem somewhere, prob picked wrong date

False

In [13]:
labelsDf.shape

(2389, 7)